In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train = pd.read_csv('/content/drive/MyDrive/영화 관객 수 예측/movies_train.csv')
test = pd.read_csv('/content/drive/MyDrive/영화 관객 수 예측/movies_test.csv')
submission = pd.read_csv('/content/drive/MyDrive/영화 관객 수 예측/submission.csv')

In [2]:
train.loc[train['dir_prev_num']==0] # 이전 작품 수가 0이면 평균 관객 수가 Nan값인 것을 알 수 있음. => 0으로 대체

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
6,길위에서,백두대간,다큐멘터리,2013-05-23,104,전체 관람가,이창재,NaN,0,32,5,53526
8,"1789, 바스티유의 연인들",유니버설픽쳐스인터내셔널코리아,뮤지컬,2014-09-18,129,전체 관람가,정성복,NaN,0,3,5,4778
9,청춘그루브,(주)두타연,드라마,2012-03-15,94,15세 관람가,변성현,NaN,0,138,3,868
10,AV 아이돌,(주) 케이알씨지,멜로/로맨스,2015-07-27,89,청소년 관람불가,조조 히데오,NaN,0,0,4,745
...,...,...,...,...,...,...,...,...,...,...,...,...
593,낭만파 남편의 편지,퍼스트런,미스터리,2013-09-12,97,청소년 관람불가,최위안,NaN,0,5,3,1132
594,오빠가 돌아왔다,(주)마운틴픽쳐스,코미디,2014-03-20,94,청소년 관람불가,노진수,NaN,0,37,5,1958
597,살인의 강,(주)마운틴픽쳐스,공포,2010-09-30,99,청소년 관람불가,김대현,NaN,0,123,4,2475
598,악의 연대기,CJ 엔터테인먼트,느와르,2015-05-14,102,15세 관람가,백운학,NaN,0,431,4,2192525


In [3]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [4]:
train.drop(['title', 'distributor', 'director', 'release_time'], axis=1, inplace=True)
test.drop(['title', 'distributor', 'director', 'release_time'], axis=1, inplace=True)

In [5]:
# 레이블 라벨링 시 이름 순으로 라벨링을 하지 않고 상관관계를 높이기 위해 크기 순으로 라벨링
train[['genre', 'box_off_num']].groupby('genre').mean().sort_values('box_off_num')

,box_off_num
genre,
뮤지컬,6.627000e+03
다큐멘터리,6.717226e+04
서스펜스,8.261100e+04
애니메이션,1.819267e+05
멜로/로맨스,4.259680e+05
미스터리,5.275482e+05
공포,5.908325e+05
드라마,6.256898e+05
코미디,1.193914e+06


In [6]:
# map함수를 이용해서 genre피쳐 인코딩
train['genre'] = train.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})
test['genre'] = test.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})

In [7]:
train[['screening_rat', 'box_off_num']].groupby('screening_rat').mean().sort_values('box_off_num')

,box_off_num
screening_rat,
전체 관람가,1.351005e+05
청소년 관람불가,3.641813e+05
12세 관람가,8.449809e+05
15세 관람가,1.247519e+06


In [8]:
train['screening_rat'] = train.screening_rat.map({'전체 관람가' : 1, '청소년 관람불가' : 2, '12세 관람가' : 3, '15세 관람가' : 4})
test['screening_rat'] = test.screening_rat.map({'전체 관람가' : 1, '청소년 관람불가' : 2, '12세 관람가' : 3, '15세 관람가' : 4})

In [9]:
train.head(3)

,genre,time,screening_rat,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,11,96,2,0.00,0,91,2,23398
1,12,130,2,1161602.50,2,387,3,7072501
2,11,123,4,220775.25,4,343,4,6959083


In [10]:
test.head(3)

,genre,time,screening_rat,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
0,12,125,2,3.005290e+05,2,304,3
1,5,113,3,3.427002e+05,4,275,3
2,8,115,3,4.206611e+06,3,419,7


In [11]:
features = ['genre', 'time', 'screening_rat', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']
target = train['box_off_num']
train = train[features]

In [12]:
train.head(3)

,genre,time,screening_rat,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
0,11,96,2,0.00,0,91,2
1,12,130,2,1161602.50,2,387,3
2,11,123,4,220775.25,4,343,4
